Rabin Karp algoritması bir pattern eşleştirme algoritmasıdır. Biz bu örnekte bir yazının içinde sözcük araması yapacağız. Ancak iki boyutlu bir sinyalde belli bir örüntü aramak gibi daha kompleks işlemlerde de kullanmak mümkündür. Bunu yapmak için , verilen küme (bu örnkete yazı) ve verilen örüntü (bu örnekte aradığımız küçük sözcük) için hash değeri hesaplayacağız. Hash değeri verilen parçaları matematiksel olarak ifade etmemizi sağlar. hash değerleri kıyaslamak tek tek harf kıyaslamaktan daha verimlidir. 

text = AAAAAAAAAH  9 A ve 1 H  
pattern = AAAAAAAAH  8A ve bir H  

eğer direkt olarak ilk sırada harf karşılaştırmaya başlarsak 8 tane a eşleştikten sonra A ve H kıyaslayıp eşit olmadığını görecek bir sonraki sıraya geçip kıayslama yapacak ve 9 karşılaştırmadan sonra eşleşmeyi bulacak.

Eğer hash hesabı yaparsak her şey daha rahat olur. 

Peki hash hesabı nedir?


Her karakteri belirten sayısal bir değer vardır 

In [ ]:
print(ord('A')) 

In [ ]:
print(ord('a'))

In [ ]:
print(ord('1')) 

stringi oluşturan karakterlerin değerini toplamak basit bir hesap çözümü

In [1]:
def simple_hash(string):

    hash_value = 0
    for char in string:
        hash_value += ord(char)  #it returns an integer representing the Unicode code of the character passed to it.
    return hash_value

In [2]:
input_string = "hello"
hash_value = simple_hash(input_string)
print("Hash value of '{}': {}".format(input_string, hash_value))

Hash value of 'hello': 532


peki ama bu bize niye yarasın ki?